In [2]:
import kagglehub
path = kagglehub.dataset_download("nodoubttome/skin-cancer9-classesisic")

In [3]:
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/skin-cancer9-classesisic


In [9]:
# 1) Install & Imports
#!pip install torch torchvision transformers scikit-image scikit-learn opencv-python kagglehub

import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from transformers import SwinForImageClassification
from torchvision import transforms
# from skimage.feature import graycomatrix, graycoprops
from skimage.feature import graycomatrix as greycomatrix, graycoprops as greycoprops
from sklearn.metrics import classification_report, accuracy_score
# import kagglehub
from torch.serialization import add_safe_globals

# 2) Download & unzip (if you haven’t already)
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# path = kagglehub.dataset_download("nodoubttome/skin-cancer9-classesisic")
# !unzip -q "{path}" -d /content/isic

class HybridGLCMSwin(nn.Module):
    def __init__(self,
                 model_name="microsoft/swin-large-patch4-window7-224-in22k",
                 num_glcm_feats=6,
                 glcm_emb_dim=128,
                 num_classes=7):
        super().__init__()
        # Swin backbone
        self.swin = SwinForImageClassification.from_pretrained(model_name)
        embed_dim = self.swin.config.hidden_size

        # MLP for GLCM
        self.glcm_mlp = nn.Sequential(
            nn.Linear(num_glcm_feats, glcm_emb_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
        )

        # final classifier on concatenated features
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim + glcm_emb_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x_img, x_glcm):
        # Swin forward
        swin_out = self.swin(pixel_values=x_img, output_hidden_states=True)
        swin_feat = swin_out.hidden_states[-1][:,0]               # [B, embed_dim]

        # GLCM embedding
        glcm_feat = self.glcm_mlp(x_glcm)                         # [B, glcm_emb_dim]

        # fuse
        joint = torch.cat([swin_feat, glcm_feat], dim=1)          # [B, embed+glcm]

        return self.classifier(joint)

class ModifiedPretrainedSwinV2(nn.Module):
    def __init__(self, model_name="microsoft/swin-large-patch4-window7-224-in22k", num_classes=7):
        super(ModifiedPretrainedSwinV2, self).__init__()
        print("Loading Swin Transformer model...")
        self.swin = SwinForImageClassification.from_pretrained(model_name)
        self.embed_dim = self.swin.config.hidden_size
        self.classifier = nn.Sequential(
            nn.Linear(self.embed_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        swin_outputs = self.swin(pixel_values=x, output_hidden_states=True)
        pooled_output = swin_outputs.hidden_states[-1][:, 0]
        outputs = self.classifier(pooled_output)
        return outputs

# 3) Define the mapping from folder names → HAM10000 labels
dir2label = {
    "actinic keratosis":          "akiec",
    "basal cell carcinoma":       "bcc",
    "pigmented benign keratosis": "bkl",
    "dermatofibroma":             "df",
    "melanoma":                   "mel",
    "nevus":                      "nv",
    "vascular lesion":            "vasc"
}

ham_classes = list(dir2label.values())

# 4) Collect (image_path, label) from Train & Test
def make_dataset(root_dir):
    data = []
    for split in ("Train", "Test"):
        split_dir = os.path.join(root_dir, split)
        for folder in os.listdir(split_dir):
            lbl = dir2label.get(folder.lower())
            if lbl is None:
                continue  # skip SCC, SK, etc.
            folder_path = os.path.join(split_dir, folder)
            for fname in os.listdir(folder_path):
                if fname.lower().endswith((".jpg",".png")):
                    data.append((os.path.join(folder_path, fname), lbl, split))
    return data

dataset = make_dataset("/kaggle/input/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration")  # adjust if your unzip path differs
print(f"Total images (filtered): {len(dataset)}")

# 5) Preprocess & GLCM helpers
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
def preprocess(img_path):
    img = cv2.imread(img_path)[:,:,::-1]
    return transform(img).unsqueeze(0).to(device), img

def compute_glcm(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    q = (gray / 32).astype(np.uint8)
    glcm = greycomatrix(q, distances=[1,2,4],
                        angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],
                        levels=8, symmetric=True, normed=True)
    feats = [greycoprops(glcm, p).mean()
             for p in ['contrast','dissimilarity','homogeneity','energy','correlation','ASM']]
    return torch.tensor(feats, dtype=torch.float32).unsqueeze(0).to(device)

# 1) Allowlist your model classes for unpickling
add_safe_globals([HybridGLCMSwin, ModifiedPretrainedSwinV2])

def load_model(path, cls):
    """
    Loads a checkpoint that may contain the full model object.
    We allow unpickling of our custom class, and set weights_only=False
    so that torch.load will actually reconstruct the object.
    """
    # map_location ensures it works on GPU or CPU
    ckpt = torch.load(path, map_location=device, weights_only=False)

    # If checkpoint is a full nn.Module instance
    if isinstance(ckpt, nn.Module):
        model = ckpt.to(device)
    # If checkpoint is a dict/state_dict
    elif isinstance(ckpt, dict):
        model = cls().to(device)
        # some people wrap with ['model_state_dict']
        if "state_dict" in ckpt:
            state_dict = ckpt["state_dict"]
        else:
            state_dict = ckpt
        model.load_state_dict(state_dict)
    else:
        raise ValueError(f"Unsupported checkpoint format: {type(ckpt)}")

    model.eval()
    return model

glcm_model = load_model("/content/drive/MyDrive/swin+GLCM_complete_trained_model_ALL_FOLDS.pth", HybridGLCMSwin)
vit_model  = load_model("/content/drive/MyDrive/swin_complete_trained_model_ALL_FOLDS.pth",   ModifiedPretrainedSwinV2)

# 7) Run inference & gather preds per split
results = {
    "Train": {"true": [], "glcm": [], "vit": []},
    "Test":  {"true": [], "glcm": [], "vit": []}
}
for img_path, label, split in dataset:
    inp, raw = preprocess(img_path)
    glcm_feats = compute_glcm(raw)
    with torch.no_grad():
        out1 = glcm_model(inp, glcm_feats)
        out2 = vit_model(inp)
    p1 = F.softmax(out1, dim=1).argmax(dim=1).item()
    p2 = F.softmax(out2, dim=1).argmax(dim=1).item()
    # reverse-map index→label
    idx2lbl = {i:ham_classes[i] for i in range(len(ham_classes))}
    results[split]["true"].append(label)
    results[split]["glcm"].append(idx2lbl[p1])
    results[split]["vit"].append(idx2lbl[p2])

# 8) Compute & print metrics for each split & model
for split in ("Train","Test"):
    print(f"\n--- {split} Split ---")
    y_true = results[split]["true"]
    for name, y_pred in (("GLCM+ViT", results[split]["glcm"]),
                         ("ViT-Only", results[split]["vit"])):
        print(f"\nModel: {name}")
        print(classification_report(
            y_true, y_pred, labels=ham_classes, digits=4))
        print("Overall accuracy:", accuracy_score(y_true, y_pred))


Total images (filtered): 2080

--- Train Split ---

Model: GLCM+ViT
              precision    recall  f1-score   support

       akiec     0.4542    1.0000    0.6247       114
         bcc     0.9210    0.9920    0.9552       376
         bkl     0.9311    0.9069    0.9189       462
          df     0.9588    0.9789    0.9688        95
         mel     0.7216    0.1598    0.2617       438
          nv     0.4510    0.6835    0.5434       357
        vasc     0.9929    1.0000    0.9964       139

    accuracy                         0.7330      1981
   macro avg     0.7758    0.8173    0.7527      1981
weighted avg     0.7746    0.7330    0.7037      1981

Overall accuracy: 0.7329631499242807

Model: ViT-Only
              precision    recall  f1-score   support

       akiec     0.4728    0.9912    0.6402       114
         bcc     0.9093    0.9867    0.9464       376
         bkl     0.8979    0.9134    0.9056       462
          df     0.9314    1.0000    0.9645        95
         m

In [11]:
# 0) Install & import kagglehub
!pip install kagglehub

import os
import kagglehub
import zipfile

# 1) Download the HAM10000 dataset
path = kagglehub.dataset_download("kmader/skin-cancer-mnist-ham10000")
print("kagglehub returned:", path)

# 2) Determine if 'path' is a directory or a zip file
if os.path.isdir(path):
    # Already unpacked
    WORK_DIR = path
    print(f"Using directory: {WORK_DIR}")
else:
    # It's a zip file → unzip it
    WORK_DIR = "/content/HAM10000"
    os.makedirs(WORK_DIR, exist_ok=True)
    with zipfile.ZipFile(path, 'r') as z:
        z.extractall(WORK_DIR)
    print(f"Unzipped to: {WORK_DIR}")

# 3) Set metadata and image directories
METADATA_CSV = os.path.join(WORK_DIR, "HAM10000_metadata.csv")
IMAGE_DIRS = [
    os.path.join(WORK_DIR, "HAM10000_images_part_1"),
    os.path.join(WORK_DIR, "HAM10000_images_part_2"),
]

# 4) Quick sanity check
print("Metadata exists:", os.path.exists(METADATA_CSV))
print("Image dir 1 exists:", os.path.isdir(IMAGE_DIRS[0]))
print("Image dir 2 exists:", os.path.isdir(IMAGE_DIRS[1]))

# 5) Load metadata & list images (example)
import pandas as pd
meta = pd.read_csv(METADATA_CSV)
print("Metadata rows:", len(meta))

# Gather a small sample to verify paths
sample = []
for bd in IMAGE_DIRS:
    for fname in os.listdir(bd)[:5]:
        sample.append(os.path.join(bd, fname))
print("Sample image paths:", sample)


kagglehub returned: /kaggle/input/skin-cancer-mnist-ham10000
Using directory: /kaggle/input/skin-cancer-mnist-ham10000
Metadata exists: True
Image dir 1 exists: True
Image dir 2 exists: True
Metadata rows: 10015
Sample image paths: ['/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1/ISIC_0028933.jpg', '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1/ISIC_0028394.jpg', '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1/ISIC_0027799.jpg', '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1/ISIC_0028100.jpg', '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1/ISIC_0027960.jpg', '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2/ISIC_0030912.jpg', '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2/ISIC_0030585.jpg', '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2/ISIC_0033697.jpg', '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2/ISIC_0030062.jpg', '/kaggle/in

In [14]:
# 1) Install & imports
# (uncomment if running in Colab)
# !pip install torch torchvision transformers scikit-image scikit-learn opencv-python pandas

import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torchvision import transforms
from transformers import SwinForImageClassification
from skimage.feature import graycomatrix, graycoprops
from sklearn.metrics import classification_report, accuracy_score
from torch.serialization import add_safe_globals

# 2) Model definitions
class HybridGLCMSwin(nn.Module):
    def __init__(self, swin_model_name="microsoft/swin-large-patch4-window7-224-in22k",
                 num_glcm_feats=6, glcm_emb_dim=128, num_classes=7):
        super().__init__()
        self.swin = SwinForImageClassification.from_pretrained(swin_model_name)
        embed_dim = self.swin.config.hidden_size
        self.glcm_mlp = nn.Sequential(
            nn.Linear(num_glcm_feats, glcm_emb_dim),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim + glcm_emb_dim, 512),
            nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
    def forward(self, x_img, x_glcm):
        swin_out = self.swin(pixel_values=x_img, output_hidden_states=True)
        swin_feat = swin_out.hidden_states[-1][:, 0]
        glcm_feat = self.glcm_mlp(x_glcm)
        joint = torch.cat([swin_feat, glcm_feat], dim=1)
        return self.classifier(joint)

class ModifiedPretrainedSwinV2(nn.Module):
    def __init__(self, model_name="microsoft/swin-large-patch4-window7-224-in22k",
                 num_classes=7):
        super().__init__()
        self.swin = SwinForImageClassification.from_pretrained(model_name)
        embed_dim = self.swin.config.hidden_size
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, 512),
            nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
    def forward(self, x):
        out = self.swin(pixel_values=x, output_hidden_states=True)
        pooled = out.hidden_states[-1][:, 0]
        return self.classifier(pooled)

# 3) Register safe globals for unpickling
add_safe_globals([HybridGLCMSwin, ModifiedPretrainedSwinV2])

# 4) Utility: load a checkpoint (full module or state-dict)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def load_model(path, cls):
    ckpt = torch.load(path, map_location=device, weights_only=False)
    if isinstance(ckpt, nn.Module):
        model = ckpt.to(device)
    elif isinstance(ckpt, dict):
        model = cls().to(device)
        sd = ckpt.get("state_dict", ckpt)
        model.load_state_dict(sd)
    else:
        raise ValueError(f"Unsupported checkpoint format: {type(ckpt)}")
    model.eval()
    return model

# 5) Preprocessing & GLCM helpers
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
def preprocess(img_path):
    img = cv2.imread(img_path)[:,:,::-1]
    return transform(img).unsqueeze(0).to(device), img

def compute_glcm(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    q = (gray / 32).astype(np.uint8)
    glcm = graycomatrix(q,
                        distances=[1,2,4],
                        angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],
                        levels=8, symmetric=True, normed=True)
    feats = [greycoprops(glcm, p).mean()
             for p in ['contrast','dissimilarity','homogeneity','energy','correlation','ASM']]
    return torch.tensor(feats, dtype=torch.float32).unsqueeze(0).to(device)

# 6) Load your models (update these to wherever you’ve placed them)
glcm_model = load_model("/content/drive/MyDrive/swin+GLCM_complete_trained_model_ALL_FOLDS.pth",
                        HybridGLCMSwin)
vit_model  = load_model("/content/drive/MyDrive/swin_complete_trained_model_ALL_FOLDS.pth",
                        ModifiedPretrainedSwinV2)

# 7) Paths for HAM10000 under Kaggle input
BASE = "/kaggle/input/skin-cancer-mnist-ham10000"
METADATA_CSV = os.path.join(BASE, "HAM10000_metadata.csv")
IMAGE_DIRS = [
    os.path.join(BASE, "HAM10000_images_part_1"),
    os.path.join(BASE, "HAM10000_images_part_2"),
]

# 8) Load metadata & build data list
meta = pd.read_csv(METADATA_CSV)
classes = ["akiec","bcc","bkl","df","mel","nv","vasc"]
cls2idx = {c:i for i,c in enumerate(classes)}
idx2cls = {i:c for i,c in enumerate(classes)}

data = []
for bd in IMAGE_DIRS:
    for fname in os.listdir(bd):
        if not fname.lower().endswith(".jpg"):
            continue
        image_id = os.path.splitext(fname)[0]
        row = meta[meta["image_id"] == image_id]
        if row.empty:
            continue
        label = row["dx"].values[0]
        data.append((os.path.join(bd, fname), label))

print(f"Total HAM10000 images: {len(data)}")

# 9) Run inference & collect preds
y_true, y_pred_glcm, y_pred_vit = [], [], []

for img_path, label in data:
    inp, raw = preprocess(img_path)
    glcm_feats = compute_glcm(raw)
    with torch.no_grad():
        out1 = glcm_model(inp, glcm_feats)
        out2 = vit_model(inp)
    p1 = F.softmax(out1, dim=1).argmax(dim=1).item()
    p2 = F.softmax(out2, dim=1).argmax(dim=1).item()

    y_true.append(label)
    y_pred_glcm.append(idx2cls[p1])
    y_pred_vit.append(idx2cls[p2])

# 10) Print per-class metrics & overall accuracy
print("=== GLCM + Swin Model ===")
print(classification_report(y_true, y_pred_glcm, labels=classes, digits=4))
print("Overall accuracy:", accuracy_score(y_true, y_pred_glcm))

print("\n=== Swin-Only Model ===")
print(classification_report(y_true, y_pred_vit, labels=classes, digits=4))
print("Overall accuracy:", accuracy_score(y_true, y_pred_vit))


Total HAM10000 images: 10015
=== GLCM + Swin Model ===
              precision    recall  f1-score   support

       akiec     0.9076    0.9908    0.9474       327
         bcc     0.9827    0.9942    0.9884       514
         bkl     0.9951    0.9217    0.9570      1099
          df     0.9741    0.9826    0.9784       115
         mel     0.9971    0.9200    0.9570      1113
          nv     0.9800    0.9990    0.9894      6705
        vasc     1.0000    1.0000    1.0000       142

    accuracy                         0.9810     10015
   macro avg     0.9766    0.9726    0.9739     10015
weighted avg     0.9815    0.9810    0.9808     10015

Overall accuracy: 0.981028457314029

=== Swin-Only Model ===
              precision    recall  f1-score   support

       akiec     0.9637    0.9755    0.9696       327
         bcc     0.9751    0.9903    0.9826       514
         bkl     0.9971    0.9336    0.9643      1099
          df     0.9744    0.9913    0.9828       115
         mel    